# **DeepCAD-RT: A cloud-based demo for DeepCAD-RT**
<img src="https://github.com/cabooster/DeepCAD-RT/blob/page/images/logo.PNG?raw=true" width = "750" height = "180" align=center />

---

## <font color=#FF8247>**1. Initialization**</font>

Install DeepCAD-RT (by default the torch GPU version is installed in COLAB notebook)

In [ ]:
!pip install deepcad

     |████████████████████████████████| 73 kB 1.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.9 MB 28.6 MB/s 
  Created wheel for deepcad: filename=deepcad-0.5.2-py3-none-any.whl size=28039 sha256=678887004846f0f106935c3132c1719725c66113bfc4b35c3404b376e4ae0f29
  Stored in directory: /root/.cache/pip/wheels/cf/c0/c9/48d1a4d134cd69bf107f2b29d7675d479341662bd1da134114
  Created wheel for gdown: filename=gdown-4.2.0-py3-none-any.whl size=14262 sha256=65f10d694f16b4d37c8ef5b745e717c19092f0b6b19ed9fe9233614cefcef303
  Stored in directory: /root/.cache/pip/wheels/8c/17/ff/58721d1fabdb87c21a0529948cf39e2be9af90ddbe4ad65944
Successfully built deepcad gdown
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gdown
    Found existing installation: gdown 

## <font color=#FF8247>**2. Check GPU support**</font>

The  implement of DeepCAD-RT need GPU support. If you do not have GPU support, please follow the following instruction.

In [ ]:
import torch

if torch.cuda.is_available():
    print('\033[1;31mGPU accessiable. Use GPU for computation.\033[0m')
    gpu_id = torch.cuda.current_device()
    total_memory = torch.cuda.get_device_properties(gpu_id).total_memory/1024/1024
    alloc_memory = torch.cuda.memory_allocated(0)/1024/1024
    print('GPU ID: ', gpu_id, '|', torch.cuda.get_device_name(), \
          '| Memory: {:.0f} MB'.format(total_memory))
    ! nvcc --version
    print('PyTorch version: ', torch.__version__)
else:
    print('\033[1;31mNo GPU support. Please enable GPUs for the notebook:\033[0m')
    print(' 1. Navigate to Edit → Notebook Settings')
    print(' 2. Select GPU from the Hardware Accelerator drop-down')

GPU accessiable. Use GPU for computation.
GPU ID:  0 | Tesla K80 | Memory: 11441 MB
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
PyTorch version:  1.10.0+cu111


## <font color=#FF8247>**Option 1: Start the training process**</font>

In [ ]:
from deepcad.train_collection import training_class
from deepcad.movie_display import display
from deepcad.utils import get_first_filename, download_demo

### Select file(s) to be processed (download if not present)

*   The `download_demo` function will download the specific file for you and return the complete path to the image and the best pretrained model files,  which will be stored in your `/content/datasets` and `/content/pth` directory, respectively. 

*   If you adapt this demo for your own data, make sure to pass the datasets folder name of your file(s).

In [ ]:
download_demo_file = True
if download_demo_file:
    file_name='fish_localbrain' # select the demo file you want to train (e.g. 'ATP_3D', 'fish_localbrain', 'NP_3D', ...)
    datasets_path, _ =download_demo(download_filename=file_name)
else:
    datasets_path = 'datasets/2RPN-1000'  # folder containing files for training

Downloading...
From: https://zenodo.org/record/5808571/files/fish_localbrain_demo.zip?download=1
To: /content/datasets/fish_localbrain_demo.zip
100%|██████████| 2.54G/2.54G [05:24<00:00, 7.84MB/s]
Downloading...
From: https://zenodo.org/record/5808571/files/fish_localbrain_E_20_Iter_6175.pth?download=1
To: /content/pth/fish_localbrain_best_model_demo/best_model.pth
100%|██████████| 4.10M/4.10M [00:07<00:00, 542kB/s]
Downloading...
From: https://zenodo.org/record/5808571/files/fish_localbrain_para.yaml?download=1
To: /content/pth/fish_localbrain_best_model_demo/best_model.yaml
100%|██████████| 337/337 [00:00<00:00, 505kB/s]


### Set up some parameters for training
Default setting shows the parameters suitable for demo file. You can change the training parameters accroding to your training data and device. For supervising the training process, you can set the flag for result storage to `True`.

In [ ]:
n_epochs = 3               # number of training epochs
GPU = '0'                 # the index of GPU you will use for computation (e.g. '0', '0,1', '0,1,2')
train_datasets_size = 3000         # datasets size for training (how many patches)
patch_xyt = 150              # the width, height, length of 3D patches (use isotropic patch size by default)
overlap_factor = 0.4           # the overlap factor between two adjacent patches
num_workers = 2              # if you use Windows system, set this to 0.

# Setup some parameters for result visualization during training period (optional)
save_test_images_per_epoch = True  # flag to select whether save inference image after each epoch in pth path

### Create a training class object with the specified parameters
You will creat a parameters object by passing all the parameters as a single dictionary. Parameters not defined in the dictionary will assume their default values. If you want to change these parameters, please refer to DeepCAD-RT Development mode.

In [ ]:
train_dict = {
    # dataset dependent parameters
    'patch_x': patch_xyt,                # you can change these params if use anisotropy patch size
    'patch_y': patch_xyt,
    'patch_t': patch_xyt,
    'overlap_factor':overlap_factor,
    'scale_factor': 1,                  # the factor for image intensity scaling
    'select_img_num': 2000,              # select the number of images used for training (use 2000 frames in colab)
    'train_datasets_size': train_datasets_size,
    'datasets_path': datasets_path,
    'pth_dir': './pth',
    
    # network related parameters
    'n_epochs': n_epochs,
    'lr': 0.00005,                       # initial learning rate
    'b1': 0.5,                          # Adam: bata1
    'b2': 0.999,                         # Adam: bata2
    'fmap': 16,                          # number of feature maps
    'GPU': GPU,
    'num_workers': num_workers,
    'visualize_images_per_epoch': False,
    'save_test_images_per_epoch': save_test_images_per_epoch
}
tc = training_class(train_dict)

Training parameters -----> 
{'overlap_factor': 0.4, 'datasets_path': 'datasets/fish_localbrain_demo', 'n_epochs': 3, 'fmap': 16, 'output_dir': './results', 'pth_dir': './pth', 'batch_size': 1, 'patch_t': 150, 'patch_x': 150, 'patch_y': 150, 'gap_y': 90, 'gap_x': 90, 'gap_t': 90, 'lr': 5e-05, 'b1': 0.5, 'b2': 0.999, 'GPU': '0', 'ngpu': 1, 'num_workers': 2, 'scale_factor': 1, 'train_datasets_size': 3000, 'select_img_num': 2000, 'test_datasize': 400, 'visualize_images_per_epoch': False, 'save_test_images_per_epoch': True}


### Start training
The training models and visual results can be downloaded in /content/pth folder.

Colab notebook needs longer time to train and test because of the limited GPU performance offered by Colab. It takes ~1100 seconds per epoch for training on GeForce RTX 3090 using default parameters.

In [ ]:
tc.run()

Image list for training -----> 
Total stack number ----->  1
Noise image name ----->  fish_localbrain.tif
Noise image shape ----->  (2000, 492, 492)
Using 1 GPU(s) for training -----> 
[Epoch 1/3] [Batch 3024/3024] [Total loss: 46218.43, L1 Loss: 203.61, L2 Loss: 92233.24] [ETA: 5:35:31] [Time cost: 10113 s]      
 Testing model of epoch 1 on the first noisy file ----->
Testing image name ----->  fish_localbrain.tif
Testing image shape ----->  (400, 492, 492)
 [Patch 100/100] [Time Cost: 79 s] [ETA: 0 s]      
 
[Epoch 2/3] [Batch 232/3024] [Total loss: 40527.80, L1 Loss: 195.08, L2 Loss: 80860.52] [ETA: 5:21:51] [Time cost: 10992 s]      

## <font color=#FF8247>**Option 2: Start the testing process**</font>

In [ ]:
from deepcad.test_collection import testing_class
from deepcad.movie_display import display
from deepcad.utils import get_first_filename, download_demo

### Select file(s) to be processed (download if not present)
*   The `download_demo` function will download the specific file for you and return the complete path to the image and the best pretrained model files,  which will be stored in your `/content/datasets` and `/content/pth` directory, respectively. 

*   If you adapt this demo for your own data, make sure to pass the datasets folder name of your file(s).

In [ ]:
download_demo_file = True
if download_demo_file:
    file_name='fish_localbrain' # select the demo file you want to test (e.g. 'ATP_3D', 'fish_localbrain', 'NP_3D', ...)
    datasets_path, denoise_model =download_demo(download_filename=file_name)
else:
    datasets_path = 'datasets/2RPN-1000'    # folder containing files for testing
    denoise_model = '2RPN_202112101128_ov_0.5' # A folder containing models to be tested

### Set up some parameters for testing
Default setting shows the parameters suitable for demo file. You can change the testing parameters accroding to your testing data and device.

In [ ]:
test_datasize = 300                   # the number of slices to be tested
GPU = '0'                        # the index of GPU you will use for computation (e.g. '0', '0,1', '0,1,2')
patch_xyt = 150                     # the width, height, length of 3D patches (use isotropic patch size by default)
overlap_factor = 0.4                   # the overlap factor between two adjacent patches
num_workers = 2                     # if you use Windows system, set this to 0.

### Create a testing class object with the specified parameters
You will create a parameters object by passing all the parameters as a single dictionary. Parameters not defined in the dictionary will assume their default values. If you want to change these parameters, please refer to DeepCAD-RT Development mode.

In [ ]:
test_dict = {
    # dataset dependent parameters
    'patch_x': patch_xyt,               # you can change these params if use anisotropy patch size
    'patch_y': patch_xyt,
    'patch_t': patch_xyt,
    'overlap_factor':overlap_factor,
    'scale_factor': 1,                 # the factor for image intensity scaling
    'test_datasize': test_datasize,
    'datasets_path': datasets_path,
    'pth_dir': './pth',                # pth file root path
    'denoise_model' : denoise_model,
    'output_dir' : './results',            # result file root path
    # network related parameters
    'fmap': 16,                     # number of feature maps
    'GPU': GPU,
    'num_workers': num_workers,
    'visualize_images_per_epoch': False,
    'save_test_images_per_epoch': True
}

tc = testing_class(test_dict)

Testing parameters -----> 
{'overlap_factor': 0.4, 'datasets_path': 'datasets/fish_localbrain_demo', 'fmap': 16, 'output_dir': './results', 'pth_dir': './pth', 'batch_size': 1, 'patch_t': 150, 'patch_x': 150, 'patch_y': 150, 'gap_y': 90, 'gap_x': 90, 'gap_t': 90, 'GPU': '0', 'ngpu': 1, 'num_workers': 2, 'scale_factor': 1, 'test_datasize': 300, 'denoise_model': 'fish_localbrain_best_model_demo', 'visualize_images_per_epoch': False, 'save_test_images_per_epoch': True}


### Start testing
The testing result can be downloaded in `/content/results` folder.

In [ ]:
tc.run()

Stacks for processing -----> 
Total stack number ----->  1
fish_localbrain.tif
Using 1 GPU(s) for testing -----> 
[Model 1/1, best_model.pth] [Stack 1/1, fish_localbrain.tif] [Patch 75/75] [Time Cost: 57 s] [ETA: 0 s]      
 Test finished. Save all results to disk.


---

## Thank you for using DeepCAD-RT!